# Section 1. Performance

In [35]:
import pandas as pd
import numpy as np

# Read the Excel file from the 'factors' sheet
df = pd.read_excel('dfa_analysis_data.xlsx', sheet_name='factors')


In [36]:
# Function to calculate performance statistics
def calculate_statistics(returns, rf_rate):
    """
    Calculate mean, volatility, Sharpe ratio, and VaR(0.05)
    
    Parameters:
    returns: Series or array of returns
    rf_rate: Series or array of risk-free rates
    
    Returns:
    Dictionary with statistics
    """
    # Convert to numpy arrays
    returns = np.array(returns)
    rf_rate = np.array(rf_rate)
    
    # Calculate mean (annualized - assuming monthly data, multiply by 12)
    mean_return = np.mean(returns) * 12
    
    # Calculate volatility (annualized - assuming monthly data, multiply by sqrt(12))
    volatility = np.std(returns, ddof=1) * np.sqrt(12)
    
    # Calculate Sharpe ratio
    excess_returns = returns - rf_rate
    sharpe_ratio = (np.mean(excess_returns) * 12) / (np.std(excess_returns, ddof=1) * np.sqrt(12))
    
    # Calculate VaR at 5% level (using historical simulation)
    var_05 = np.percentile(returns, 5)
    
    return {
        'Mean': mean_return,
        'Volatility': volatility,
        'Sharpe': sharpe_ratio,
        'VaR(0.05)': var_05
    }

In [37]:
date_col = df.columns[0]
df[date_col] = pd.to_datetime(df[date_col])
df = df.sort_values(date_col)


In [38]:
period1 = df[df[date_col] <= '1980-12-31']
period2 = df[(df[date_col] >= '1981-01-01') & (df[date_col] <= '2001-12-31')]
period3 = df[df[date_col] >= '2002-01-01']


In [39]:
# Calculate statistics for all three periods and all factors
def analyze_all_periods(period_data, period_name, factors, rf_col):
    """
    Analyze all factors for a given period
    """
    print(f"\n{'='*80}")
    print(f"{period_name}")
    print(f"{'='*80}")
    
    results = {}
    
    for factor in factors:
        stats = calculate_statistics(period_data[factor], period_data[rf_col])
        results[factor] = stats
    
    return results

In [40]:
mkt_col = 'Mkt-RF'
smb_col = 'SMB'
hml_col = 'HML'
rf_col = 'RF'

In [41]:
# Run analysis for all three periods
if all([mkt_col, smb_col, hml_col, rf_col]):
    factors_dict = {'mkt': mkt_col, 'smb': smb_col, 'hml': hml_col}
    
    results_p1 = analyze_all_periods(period1, "Period 1: Beginning - 1980", 
                                      [mkt_col, smb_col, hml_col], rf_col)
    results_p2 = analyze_all_periods(period2, "Period 2: 1981 - 2001", 
                                      [mkt_col, smb_col, hml_col], rf_col)
    results_p3 = analyze_all_periods(period3, "Period 3: 2002 - End", 
                                      [mkt_col, smb_col, hml_col], rf_col)
else:
    print("Error: Could not identify all required columns. Please check column names.")


Period 1: Beginning - 1980

Period 2: 1981 - 2001

Period 3: 2002 - End


In [42]:
# Create a summary table
def create_summary_table(results_dict, period_names):
    """
    Create a formatted summary table of all results
    """
    summary_data = []
    
    for period_name, results in zip(period_names, results_dict):
        for factor_name, stats in results.items():
            summary_data.append({
                'Period': period_name,
                'Factor': factor_name,
                'Mean': stats['Mean'],
                'Volatility': stats['Volatility'],
                'Sharpe': stats['Sharpe'],
                'VaR(0.05)': stats['VaR(0.05)']
            })
    
    summary_df = pd.DataFrame(summary_data)
    return summary_df

if all([mkt_col, smb_col, hml_col, rf_col]):
    period_names = ['Beginning - 1980', '1981 - 2001', '2002 - End']
    summary_df = create_summary_table([results_p1, results_p2, results_p3], period_names)
    
    print("\n" + "="*80)
    print("SUMMARY TABLE")
    print("="*80)
    print(summary_df.to_string(index=False))
    


SUMMARY TABLE
          Period Factor      Mean  Volatility    Sharpe  VaR(0.05)
Beginning - 1980 Mkt-RF  0.080958    0.204988  0.258799  -0.084090
Beginning - 1980    SMB  0.033914    0.114277  0.053857  -0.041875
Beginning - 1980    HML  0.050321    0.134228  0.167743  -0.044245
     1981 - 2001 Mkt-RF  0.077852    0.157183  0.093351  -0.064135
     1981 - 2001    SMB -0.002014    0.117260 -0.552021  -0.045880
     1981 - 2001    HML  0.064557    0.109863  0.013491  -0.041640
      2002 - End Mkt-RF  0.091306    0.153529  0.490639  -0.077265
      2002 - End    SMB  0.007931    0.088448 -0.088683  -0.039195
      2002 - End    HML  0.001204    0.106442 -0.137055  -0.041485


1) from the results, market factor has a premium in each subsample, the size factor has a negative premiums in 1981-2001 and 2002-end. value factor has a postive premium in each subsample. 

2) yes, the size factor was positive 3.39% before 1980 and it went down to -0.2% during 1980-2001 and slighly recovered to 0.79% after 2002. 

3) the value factor increases during 1990s, from 5.03% to 6.45%


4) market factor is still pretty strong, but size factor and value factor's premium have come down significantly. 

# Section 2. Correlation Analysis

In [43]:
# Calculate correlation matrices for each subsample
factors_list = [mkt_col, smb_col, hml_col]

print("="*80)
print("CORRELATION MATRICES FOR THE THREE FACTORS")
print("="*80)

# Period 1: Beginning - 1980
corr_p1 = period1[factors_list].corr()
print("\nPeriod 1: Beginning - 1980")
print(corr_p1)

# Period 2: 1981 - 2001
corr_p2 = period2[factors_list].corr()
print("\nPeriod 2: 1981 - 2001")
print(corr_p2)

# Period 3: 2002 - End
corr_p3 = period3[factors_list].corr()
print("\nPeriod 3: 2002 - End")
print(corr_p3)

CORRELATION MATRICES FOR THE THREE FACTORS

Period 1: Beginning - 1980
          Mkt-RF       SMB       HML
Mkt-RF  1.000000  0.374234  0.442341
SMB     0.374234  1.000000  0.313397
HML     0.442341  0.313397  1.000000

Period 2: 1981 - 2001
          Mkt-RF       SMB       HML
Mkt-RF  1.000000  0.158652 -0.529912
SMB     0.158652  1.000000 -0.445451
HML    -0.529912 -0.445451  1.000000

Period 3: 2002 - End
          Mkt-RF       SMB       HML
Mkt-RF  1.000000  0.316034  0.109671
SMB     0.316034  1.000000  0.116227
HML     0.109671  0.116227  1.000000


1)  all of those correlations are positive but not super big, so the construction succeeds in keeping it small

2) after 1980 there's strong negative correlation between mkt factor and value factor, value factor and size factor. 
so even though each subperiods correlation is alright, it looks like the factors' correlation varies quite a bit, which means the construction is not that stable